In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random
import cv2
import pywt
import keras
import string
import statsmodels.api as sm 
import matplotlib.gridspec as gridspec
import itertools
import missingno as msno
import folium
from folium.plugins import HeatMap
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

from cv2 import IMREAD_COLOR,IMREAD_UNCHANGED
from scipy.ndimage import variance
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import laplace
from skimage.transform import resize
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder
from mlxtend.plotting import plot_decision_regions
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from mlxtend.classifier import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [2]:
train = pd.read_csv('/kaggle/input/ods-huawei/new_train.csv')
f_train = pd.read_csv('/kaggle/input/ods-huawei/feature_train.csv')
train = pd.concat([f_train['rate'], train], axis=1)

In [3]:
non_numeric_columns = train.select_dtypes(include=['float64', 'int64']).columns

In [4]:
from sklearn.model_selection import KFold

y = train['rate']
X = train[non_numeric_columns].drop(columns = 'rate')

kf = KFold(n_splits=25)
kf.get_n_splits(X)

3

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.model_selection import cross_val_score
from tqdm.notebook import tqdm

classifiers = [
    LogisticRegression(multi_class='multinomial', solver='newton-cg'),
    RandomForestClassifier(max_depth=50, n_estimators=500),
    SVC(random_state=0, probability=True),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    GradientBoostingClassifier(max_depth=5, n_estimators=500),
    XGBClassifier(booster='gbtree', n_estimators=300),
    CatBoostClassifier(iterations=300, loss_function='MultiClass'),
    ExtraTreesClassifier(max_depth=50, criterion='gini', n_estimators=500),
    AdaBoostClassifier(base_estimator=ExtraTreesClassifier(max_depth=50, criterion='gini', n_estimators=500), n_estimators=50),
    LGBMClassifier(learning_rate=1, boosting_type='gbdt', objective='multiclass')
]

labels = ['Логистическая регрессия', 'Случайный лес', 'Метод опорных векторов', 'KNN', 'Деревья решений',
          'Ada Boost', 'Gradient Boosting', 'XgBoost', 'Cat Boost', 'Extra Trees', 'LGBM']

clfs = []

for clf, lab in zip(classifiers, labels):
    precision_scores = []
    f1_scores = []
    print(lab)
    with tqdm(total=kf.get_n_splits(X)) as pbar:
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.values[train_index], X.values[test_index]
            y_train, y_test = y.values[train_index], y.values[test_index]
            
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            
            precision = precision_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            
            precision_scores.append(precision)
            f1_scores.append(f1)
            
            pbar.set_description(f'F1: {f1:.2f}')
            pbar.update(1)

    mean_precision = sum(precision_scores) / len(precision_scores)
    mean_f1 = sum(f1_scores) / len(f1_scores)
    
    clfs.append((lab, mean_precision, mean_f1))
    print(f'{lab}: Macro Precision = {mean_precision:.2f}, Macro F1 = {mean_f1:.2f}')


# Вывести результаты оценки всех моделей
for clf, precision, f1 in clfs:
    print(f'{clf}: Macro Precision = {precision.mean():.2f}, Macro F1 = {f1.mean():.2f}')


LGBM


  0%|          | 0/3 [00:00<?, ?it/s]

LGBM: Macro Precision = 0.36, Macro F1 = 0.35
LGBM: Macro Precision = 0.36, Macro F1 = 0.35


In [6]:
df_score = pd.DataFrame({
    "Mean Score": clfs,
    "Algorithm": labels,
})

fig = px.bar(data_frame=df_score, x='Mean Score', y='Algorithm', color='Mean Score', title='Оценки моделей')
fig.show()
